In [ ]:
import random
import numpy as np
import torch
from main_ae import AgeModel
from main_ae import START_AGE, END_AGE, NUM_AGE_GROUPS, LAMBDA_1, LAMBDA_2

In [ ]:
NUM_AGE_GROUPS = 9
VALIDATION_RATE = 0.1
num_ages = END_AGE - START_AGE + 1

random.seed(2019)
np.random.seed(2019)
torch.manual_seed(2019)

# AIHub dataset


In [ ]:
import json
import pandas as pd

In [ ]:
from fpt.data import join_face_df
from fpt.path import DTFR

DATA_CATEGORY = "aihub_family"
face_df = join_face_df(DTFR, DATA_CATEGORY)

In [ ]:
RANGE_TO_MEDIAN = {
    "a": (1 + 6) / 2,
    "b": (7 + 12) / 2,
    "c": (13 + 19) / 2,
    "d": (20 + 30) / 2,
    "e": (31 + 45) / 2,
    "f": (46 + 55) / 2,
    "g": (56 + 66) / 2,
    "h": (67 + 80) / 2,
    "above": 90,
}
AGE_GROUPS = ["a", "b", "c", "d", "e", "f", "g", "h", "above"]


def age_to_age_groups(age):
    if age <= 6:
        return "a"
    if age <= 12:
        return "b"
    if age <= 19:
        return "c"
    if age <= 30:
        return "d"
    if age <= 45:
        return "e"
    if age <= 55:
        return "f"
    if age <= 66:
        return "g"
    if age <= 80:
        return "h"
    return "above"

GROUP_TO_INDEX = {group: index for index, group in enumerate(AGE_GROUPS)}

# NIA Age Model

## Model


In [ ]:
model = AgeModel(num_ages, NUM_AGE_GROUPS)  # age_pred, age_group_pred

## Dataloader


In [ ]:
from data import NiaDataset
import numpy as np
from PIL import Image
import os


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision


In [ ]:
train_meta_path = "nia_cropped/train_0.npy"
test_meta_path = "nia_cropped/test_0.npy"


In [ ]:
batch_size = 16


### NiaDataset


#### Train dataloader


In [ ]:
transforms_train = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomApply(
            [
                torchvision.transforms.RandomAffine(degrees=10, shear=16),
                torchvision.transforms.RandomHorizontalFlip(p=1.0),
            ],
            p=0.5,
        ),
        torchvision.transforms.Resize((256, 256)),
        torchvision.transforms.RandomCrop((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)


In [ ]:
train_gen = NiaDataset(train_meta_path, transforms_train)


In [ ]:
train_loader = DataLoader(
    dataset=train_gen,
    batch_size=batch_size,
    shuffle=True,
    pin_memory=True,
    num_workers=0,
)


In [ ]:
train_iter = iter(train_loader)


In [ ]:
sample = next(train_iter)
sample.keys()


In [ ]:
for key in sample.keys():
    print(f"{key}:\t{sample[key][0]}")

In [ ]:
face_df.key

In [ ]:
idx = 1
file_name, _ = os.path.splitext(sample['file'][idx])
file_name in face_df.key

#### Validation dataloader


In [ ]:
transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)
val_gen = NiaDataset(test_meta_path, transforms)
val_loader = DataLoader(
    val_gen, batch_size=1, shuffle=False, pin_memory=True, num_workers=0
)


## Optimizer


In [ ]:
from torch import optim
import torch
from torch.optim import lr_scheduler


In [ ]:
LAMBDA_1 = 0.2
LAMBDA_2 = 0.05
START_AGE = 0
END_AGE = 90
learning_rate = 1e-3
epoch = 2


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[5, 8, 9], gamma=0.1)


## Loss function


In [ ]:
from mean_variance_loss import MeanVarianceLoss


In [ ]:
criterion1 = MeanVarianceLoss(LAMBDA_1, LAMBDA_2, START_AGE, END_AGE).cuda
criterion2 = torch.nn.CrossEntropyLoss().cuda()


## Train


In [ ]:
from main_ae import train_softmax, evaluate_softmax


In [ ]:
def train_softmax(train_loader, model, criterion2, optimizer, epoch, result_directory):
    model.cuda().train()
    running_loss = 0.0
    running_softmax_loss = 0.0
    interval = 1
    for i, sample in enumerate(train_loader):
        images = sample["image"].cuda()
        labels = sample["age_class"].cuda()
        _, output = model(images)
        loss = criterion2(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.data
        if (i + 1) % interval == 0:
            print("[%d, %5d] loss: %.3f" % (epoch, i, running_loss / interval))
            with open(os.path.join(result_directory, "log"), "a") as f:
                f.write("[%d, %5d] loss: %.3f\n" % (epoch, i, running_loss / interval))
            running_loss = 0.0


In [ ]:
for epoch in range(2):
    break
    train_softmax(train_loader, model, criterion2, optimizer, epoch, "result")
    loss_val, mae = evaluate_softmax(val_loader, model, criterion2)
    scheduler.step()


In [ ]:
# loss_val, mae


# Multi-task Model


## Face Recognition


In [ ]:
from arcface_torch.losses import CombinedMarginLoss
from arcface_torch.configs.aihub_r50_onegpu import config as aihub_config
from arcface_torch.configs.base import config as cfg

cfg.update(aihub_config)
cfg.output = "work_dirs/aihub_r50_onegpu"

In [ ]:
margin_loss = CombinedMarginLoss(
    64,
    cfg.margin_list[0],
    cfg.margin_list[1],
    cfg.margin_list[2],
    cfg.interclass_filtering_threshold,
)

## AIHubDataset

In [ ]:
from facenet.datasets.AIHubDataset import AIHubDataset
# from to
from torchvision.datasets import ImageFolder
import torchvision

In [ ]:
root_dir = '/home/jupyter/data/face-image/train_aihub_family'
image_size = 112
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]
face_age_transform_train = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

In [ ]:
train_set = ImageFolder(root_dir, face_age_transform_train)
next(iter(train_set))

## Multi task Dataset


In [ ]:
import os
import torch
from torch.utils.data import Dataset
import torchvision
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from easydict import EasyDict as edict

### Train dataset

In [ ]:
root_dir = '/home/jupyter/data/face-image/train_aihub_family'
image_size = 112
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]
face_age_transform_train = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize(size=(image_size, image_size)),
        torchvision.transforms.RandomHorizontalFlip(),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

In [ ]:
face_age_transforms_train_nia = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.RandomApply(
            [
                torchvision.transforms.RandomAffine(degrees=10, shear=16),
                torchvision.transforms.RandomHorizontalFlip(p=1.0),
            ],
            p=0.5,
        ),
        torchvision.transforms.Resize((256, 256)),
        torchvision.transforms.RandomCrop((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)

In [ ]:
class FaceAgeDataset(Dataset):
    def __init__(self, root_dir, face_df, transform):
        self.face_dataset = ImageFolder(root=root_dir, transform=transform)
        self.face_df = face_df
        self.class_to_idx = self.face_dataset.class_to_idx

    def __len__(self):
        return len(self.face_dataset)

    def __getitem__(self, index):
        image, face_label = self.face_dataset[index]
        path, _ = self.face_dataset.samples[index]
        *_, key = os.path.splitext(path)[0].split("/")
        row = face_df.loc[key]
        sample = edict({
            "image": image,
            "age": row.age,
            "age_class": GROUP_TO_INDEX[row.age_group],
            "file": path,
            "data_type": row.category,
            "family_id": row.family_id,
            "personal_id": row.target,
            "face_label": face_label,
            "key": key,
        })
        return sample

In [ ]:
root_dir = "/home/jupyter/data/face-image/train_aihub_family"
face_age_dataset = FaceAgeDataset(
    root_dir,
    face_df,
    face_age_transform_train,
)
iterator = iter(face_age_dataset)

In [ ]:
sample = next(iterator)
[print(f"{k}: {v}") for k, v in sample.items()]
pass

### Validation dataset

In [ ]:
from facenet.datasets.AIHubDataset import AIHubDataset

In [ ]:
valid_root_dir = "/home/jupyter/data/face-image/valid_aihub_family"
num_workers = 4
lfw_batch_size = 200
image_size = 112
aihub_mean = [0.5444, 0.4335, 0.3800]
aihub_std = [0.2672, 0.2295, 0.2156]

face_age_transform_valid = transforms.Compose(
    [
        transforms.Resize(size=(image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=aihub_mean, std=aihub_std),
    ]
)

In [ ]:
face_age_valid_dataset = FaceAgeDataset(
    valid_root_dir,
    face_df,
    face_age_transform_valid,
)

In [ ]:
valid_iter = iter(face_age_valid_dataset)

In [ ]:
valid_sample = next(valid_iter)
valid_sample.family_id

### Verification

In [ ]:
face_age_verification_valid_dataset = AIHubDataset(
    dir=valid_root_dir,
    pairs_path="data/pairs/valid/pairs_Age.txt",
    transform=face_age_transform_valid,
)

In [ ]:
pairs_dataset = iter(face_age_verification_valid_dataset)

In [ ]:
verification_valid_sample = next(pairs_dataset)  # img1, img2, issame

## DataLoader

### Train dataloader

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate


def custom_collate(batch):
    data = {
        key: default_collate([d[key] for d in batch])
        for key in batch[0]
        if key != "age"
    }
    age = torch.tensor(
        [d["age"] for d in batch], dtype=torch.float32
    )  # 'age' 필드를 텐서로 변환합니다.
    data["age"] = age
    return data


batch_size = 16
face_age_loader = DataLoader(
    dataset=face_age_dataset,
    batch_size=batch_size,
    shuffle=True,
)
train_iter = iter(face_age_loader)
sample = next(train_iter)

In [ ]:
print(sample.family_id)
print(sample.age_class)
print(sample.personal_id)
print(sample.face_label)
sample.image.shape, sample.age

### Valid dataloader

In [ ]:
batch_size = 16
face_age_loader = DataLoader(
    dataset=face_age_dataset,
    batch_size=batch_size,
    shuffle=True,
)
train_iter = iter(face_age_loader)
sample = next(train_iter)

In [ ]:
face_age_valid_loader = DataLoader(
    face_age_valid_dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=0,
)

#### verification

In [ ]:
aihub_dataloader = torch.utils.data.DataLoader(
    dataset=face_age_verification_valid_dataset,
    batch_size=lfw_batch_size,
    num_workers=num_workers,
    shuffle=False,
)

In [ ]:
verification_iter = iter(aihub_dataloader)

In [ ]:
next(verification_iter)

## Model

### nia_age

In [ ]:
from main_ae import AgeModel, Embedding
from main_ae import END_AGE, START_AGE, NUM_AGE_GROUPS
NUM_AGES = END_AGE - START_AGE + 1

In [ ]:
model = AgeModel(NUM_AGES, NUM_AGE_GROUPS)
model = model.cuda()

In [ ]:
images = sample.image.cuda()

In [ ]:
age_pred, age_group_pred = model(images)
age_pred.shape, age_group_pred.shape

In [ ]:
embedding = Embedding()
embedding = embedding.cuda()
embedding = embedding.eval()

In [ ]:
nia_embedding_result = embedding(images)
nia_embedding_result.shape

### ArcFace

In [ ]:
from arcface_torch.backbones import get_model
from arcface_torch.configs.aihub_r50_onegpu import config as aihub_config
from arcface_torch.configs.base import config as cfg

cfg.update(aihub_config)
cfg.output = "work_dirs/aihub_r50_onegpu"

In [ ]:
backbone = get_model(
    cfg.network,
    dropout=0.0,
    fp16=False,
    num_features=512,
).cuda()

In [ ]:
model_weights = torch.load(
    f"/home/jupyter/family-photo-tree/utils/model/arcface/{cfg.network}/backbone.pth"
)
backbone.load_state_dict(model_weights)

In [ ]:
backbone = backbone.eval()

In [ ]:
arcface_result = backbone(images)
arcface_result.shape

### Age Classifier with ArcFace

In [ ]:
from main_ae import AgeClassifier

In [ ]:
age_classifier = AgeClassifier(NUM_AGES, NUM_AGE_GROUPS).cuda()

In [ ]:
pred_age_with_arcface, pred_age_group_with_arcface = age_classifier(arcface_result)
pred_age_with_arcface.shape, pred_age_group_with_arcface.shape

## Loss Function


### CrossEntropyLoss

In [ ]:
import torch

In [ ]:
cross_entropy_loss_nia = torch.nn.CrossEntropyLoss().cuda()
age_loss = cross_entropy_loss_nia(age_pred, sample.age.cuda())
age_group_loss = cross_entropy_loss_nia(age_group_pred, sample.age_class.cuda()) 

### Mean-Variance Loss

In [ ]:
from mean_variance_loss import MeanVarianceLoss

In [ ]:
mean_variance_loss = MeanVarianceLoss(LAMBDA_1, LAMBDA_2, START_AGE, END_AGE).cuda()
mean_loss, variance_loss = mean_variance_loss(age_pred, sample.age)

In [ ]:
total_loss_nia = mean_loss + variance_loss + age_loss + 10 * age_group_loss
total_loss_nia

### CombinedMarginLoss | ArcFace | CosFace

In [ ]:
from arcface_torch.losses import CombinedMarginLoss, ArcFace, CosFace
from torch.nn.functional import normalize, linear
from torch.nn import CrossEntropyLoss

num_classes = len(face_age_dataset.class_to_idx)

In [ ]:
margin_loss = CombinedMarginLoss(
    64,
    cfg.margin_list[0],
    cfg.margin_list[1],
    cfg.margin_list[2],
    cfg.interclass_filtering_threshold,
)
arc_face_loss = ArcFace()
cos_face_loss = CosFace()
cross_entropy_loss_arcface = torch.nn.CrossEntropyLoss()

In [ ]:
# labels
labels = sample.face_label.cuda()
labels.squeeze_()
labels = labels.long()
labels = labels.view(-1, 1)

In [ ]:
# embeddings
norm_embeddings = normalize(arcface_result)

In [ ]:
# weight
weight = torch.nn.Parameter(torch.normal(0, 0.01, (num_classes, 512))).cuda()
norm_weight_activated = normalize(weight)
norm_weight_activated.shape

In [ ]:
# logits
logits = linear(norm_embeddings, norm_weight_activated)
logits = logits.clamp(-1, 1)
# softmax = margin_loss(logits, labels)
# softmax = arc_face_loss(logits, labels)
softmax = cos_face_loss(logits, labels)

In [ ]:
# loss
loss_arcface = cross_entropy_loss_arcface(softmax, labels.flatten())
loss_arcface

## Optimizer

In [ ]:
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR
from arcface_torch.lr_scheduler import PolyScheduler

### nia_age

In [ ]:
optimizer_nia = Adam(model.parameters(), lr=0.001)
scheduler_nia = StepLR(optimizer_nia, 5)
optimizer_nia.zero_grad()
total_loss_nia.backward()
optimizer_nia.step()
scheduler_nia.step()

### ArcFace

In [ ]:
momentum = 0.9  #
weight_decay = 5e-4  #
lr = 0.02

opt = torch.optim.SGD(
    params=[
        {"params": backbone.parameters()},
        {"params": margin_loss.parameters()},
    ],
    lr=lr,
    momentum=momentum,
    weight_decay=weight_decay,
)

In [ ]:
num_epoch = 2
world_size = 1
cfg.total_batch_size = cfg.batch_size * world_size
cfg.warmup_step = cfg.num_image // cfg.total_batch_size * cfg.warmup_epoch
cfg.total_step = cfg.num_image // cfg.total_batch_size * num_epoch

lr_scheduler = PolyScheduler(
    optimizer=opt,
    base_lr=lr,
    max_steps=cfg.total_step,
    warmup_steps=cfg.warmup_step,
    last_epoch=-1,
)

In [ ]:
opt.zero_grad()
loss_arcface.backward()
opt.step()
lr_scheduler.step()

In [ ]:
from torchviz import make_dot

# 시각화
dot = make_dot(total_loss_nia, params=dict(model.named_parameters()))
dot.format = "pdf"  # 'png'로 형식 변경
dot.render("graph")  # 'graph.png' 파일로 저장

## Train

In [ ]:
from tqdm import tqdm

### NIA Age
#### train softmax

In [ ]:
running_loss = 0.
interval = 10
for index, sample in enumerate(face_age_loader):
    images = sample.image.cuda()
    labels = sample.age.cuda()
    age_pred, age_group_red = model(images)
    age_loss = cross_entropy_loss_nia(age_pred, labels)
    optimizer_nia.zero_grad()
    age_loss.backward()
    optimizer_nia.step()
    running_loss += age_loss.data
    if (index + 1) % interval == 0:
        print(f"loss: {running_loss/interval:.3f}")
        running_loss = 0.0

#### Age & Age_group

In [ ]:
running_loss = 0.0
interval = 10

for index, sample in enumerate(face_age_loader):
    images = sample.image.cuda()
    labels = sample.age.cuda()
    age_group_labels = sample.age_class.cuda()
    age_pred, age_group_pred = model(images)
    
    dta = np.array(sample.data_type)
    age_sample_indices = dta != "Age"
    age_pred = age_pred[age_sample_indices]
    labels = labels[age_sample_indices]
    
    mean_loss, variance_loss = mean_variance_loss(age_pred, labels)
    age_softmax_loss = cross_entropy_loss_nia(age_pred, labels)
    mean_loss, variance_loss, age_softmax_loss
    
    age_group_pred = age_group_pred[~age_sample_indices]
    age_group_labels = age_group_labels[~age_sample_indices]
    age_group_softmax_loss = cross_entropy_loss_nia(age_group_pred, age_group_labels)
    
    total_loss = mean_loss + variance_loss + age_softmax_loss + 10 * age_group_softmax_loss
    
    optimizer_nia.zero_grad()
    total_loss.backward()
    optimizer_nia.step()
    
    running_loss += total_loss.data
    if (index + 1) % interval == 0:
        print(f"loss: {running_loss/interval:.3f}")
        running_loss = 0.0
    # break

## Evaluation

In [ ]:
from PIL import Image
from tqdm import tqdm
import numpy as np
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision import transforms
import time

In [ ]:
from data import NiaDataset
from main_ae import evaluate_1, AgeModel
from main_ae import START_AGE, END_AGE, NUM_AGE_GROUPS

In [ ]:
save_model_path = "/home/jongphago/nia_age/result_model/model_0"
test_meta_path = "nia_cropped/test_0.npy"
val_gen = NiaDataset(test_meta_path, transforms)
val_loader = DataLoader(
    val_gen,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=0,
)

In [ ]:
model = AgeModel(END_AGE - START_AGE + 1, NUM_AGE_GROUPS)
model.cuda()
model.load_state_dict(torch.load(save_model_path))

In [ ]:
nia_valid_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)
val_gen = NiaDataset(test_meta_path, nia_valid_transforms)
val_loader = DataLoader(
    val_gen,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=0,
)

In [ ]:
mae, ag_acc, log_dict = evaluate_1(val_loader, model)  # ag_acc 0.6746
print("Mae---", mae)

### `evaluate_1`

In [ ]:
model_params = torch.load(save_model_path)
model.load_state_dict(model_params)

In [ ]:
from PIL import Image


def display_sample(sample):
    # DataLoader에서 첫 번째 이미지를 가져옵니다.
    image_tensor = sample["image"].cpu().numpy()

    # 이미지 텐서의 차원을 줄입니다.
    image_array = np.squeeze(image_tensor)

    # 배열의 축을 재배치합니다 (채널을 마지막 축으로 이동).
    image_array = image_array.transpose(1, 2, 0)

    # [0, 1] 범위에서 [0, 255] 범위로 스케일링합니다.
    image_array = (image_array * 255).astype(np.uint8)

    # 이미지 배열을 PIL 이미지로 변환합니다.
    image = Image.fromarray(image_array)

    # 이미지 크기를 조절합니다.
    resized_image = image.resize((80, 80))

    # 이미지를 표시합니다.
    display(resized_image)

In [ ]:
def my_evaluate(val_loader, model, task=("kinship")):
    log_dict = dict()
    log_dict["No."] = []
    log_dict["image_path"] = []
    log_dict["data_type"] = []
    log_dict["GT_age"] = []
    log_dict["predicted_age"] = []
    log_dict["GT_age_group"] = []
    log_dict["predicted_age_group"] = []
    log_dict["absolute_error"] = []
    log_dict["mean_absolute_error"] = []
    log_dict["age_group_classification_accuracy"] = []

    model.eval()
    ae_sum = 0.0
    kinship_counter = 0
    age_counter = 0
    age_group_success = 0
    tics = time.time()
    with torch.no_grad():
        for i, sample in tqdm(enumerate(val_loader)):
            image = sample["image"].cuda()
            output, output2 = model(image)
            m = torch.nn.Softmax(dim=1)
            output_softmax = m(output)

            a = torch.arange(START_AGE, END_AGE + 1, dtype=torch.float32).cuda()
            mean = (output_softmax * a).sum(1, keepdim=True).cpu().data.numpy()
            pred = np.around(mean)[0][0]
            pred_age_group = np.argmax(output2.cpu().data.numpy())
            gt_age_group = sample["age_class"].cpu().item()
            gt_age = sample["age"].cpu().item()
            ae = np.absolute(pred - gt_age)

            filename = sample["file"][0]
            log_dict["No."].append(i)
            log_dict["image_path"].append(filename)
            data_type = sample["data_type"][0]
            log_dict["data_type"].append(data_type)
            if data_type in task:
                # display_sample(sample)
                log_dict["GT_age"].append(gt_age)
                log_dict["predicted_age"].append(pred)
                log_dict["absolute_error"].append(ae)
                log_dict["GT_age_group"].append("-")
                log_dict["predicted_age_group"].append("-")
                ae_sum += ae
                kinship_counter += 1
            else:
                # display_sample(sample)
                log_dict["GT_age"].append("-")
                log_dict["predicted_age"].append("-")
                log_dict["absolute_error"].append("-")
                log_dict["GT_age_group"].append(gt_age_group)
                log_dict["predicted_age_group"].append(pred_age_group)
                if gt_age_group == pred_age_group:
                    age_group_success += 1
                age_counter += 1
            mae = ae_sum / kinship_counter if kinship_counter > 0 else "-"
            ag_acc = age_group_success / age_counter if age_counter > 0 else "-"
            log_dict["mean_absolute_error"].append(mae)
            log_dict["age_group_classification_accuracy"].append(ag_acc)
            
    print("# validation ----", len(val_loader))
    return ae_sum / kinship_counter, age_group_success / age_counter, log_dict

In [ ]:
mae, accuarcy, log_dict = my_evaluate(val_loader, model)  # 2m 10s
print(mae, accuarcy, log_dict)

In [ ]:
nia_valid_transforms_for_acrface = torchvision.transforms.Compose(
    [
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
    ]
)

nia_face_age_valid_dataset = FaceAgeDataset(
    valid_root_dir,
    face_df,
    nia_valid_transforms_for_acrface,
)

nia_face_age_valid_loader = DataLoader(
    nia_face_age_valid_dataset,
    batch_size=1,
    shuffle=False,
    pin_memory=True,
    num_workers=0,
)

In [ ]:
# ag_acc 0.72
my_evaluate(nia_face_age_valid_loader, model, ("Individuals", "Family"))

### `evaluate_softmax`

In [ ]:
from tqdm import tqdm
from main_ae import AgeModel

In [ ]:
save_model_path = "/home/jongphago/nia_age/result_model/model_0"
model = AgeModel(END_AGE - START_AGE + 1, NUM_AGE_GROUPS)
model.cuda()
model.load_state_dict(torch.load(save_model_path))

In [ ]:
def evaluate_softmax(val_loader, model, criterion2):
    model.cuda()
    model.eval()
    loss_val = 0.0
    softmax_loss_val = 0.0
    mae = 0.0
    with torch.no_grad():
        for i, sample in tqdm(enumerate(val_loader)):
            image = sample["image"].cuda()
            label = sample["age"].cuda()
            age_class = sample["age_class"].cuda()
            age_pred, age_group_pred = model(image)
            data_type = sample["data_type"][0]
            if data_type in ("Individuals", "Family"):
                loss = criterion2(age_pred, label) + criterion2(age_group_pred, age_class)
            else:
                loss = criterion2(age_group_pred, age_class)
            loss_val += loss.data
            m = torch.nn.Softmax(dim=1)
            output_softmax = m(age_pred)
            a = torch.arange(START_AGE, END_AGE + 1, dtype=torch.float32).cuda()
            mean = (output_softmax * a).sum(1, keepdim=True).cpu().data.numpy()
            pred = np.around(mean)
            # print("-------pred", pred.astype(int).item(), end='\t')
            # print("-------label", label.cpu().data.numpy().item())
            mae += np.absolute(pred - sample["age"].cpu().data.numpy())
    return loss_val / len(val_loader), mae / len(val_loader)

In [ ]:
cross_entropy_loss = torch.nn.CrossEntropyLoss().cuda()

In [ ]:
evaluate_softmax(
    nia_face_age_valid_loader,
    model,
    cross_entropy_loss,
)  # mae 3.77

### `validate_aihub`